In [1]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

class FeatureExtractorFineTuner:
    def __init__(self, input_shape, num_classes, learning_rates):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rates = learning_rates
        self.feature_extractor = self.create_feature_extractor()
        self.fine_tuning_started = False
    
    def create_feature_extractor(self):
        # Load the DenseNet model pre-trained on ImageNet
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=self.input_shape)
        
        # Freeze the convolutional layer blocks and flatten layer
        for layer in base_model.layers:
            if 'conv' in layer.name or 'pool' in layer.name or 'flatten' in layer.name:
                layer.trainable = False
        
        # Get the output of the last convolutional layer
        output = base_model.output
        
        # Add a global average pooling layer
        output = tf.keras.layers.GlobalAveragePooling2D()(output)
        
        # Add a fully connected layer with softmax activation for classification
        output = tf.keras.layers.Dense(self.num_classes, activation='softmax')(output)
        
        # Create the feature extraction model
        feature_extractor = Model(inputs=base_model.input, outputs=output)
        
        return feature_extractor
    
    def fine_tune_model(self):
        # Get the total number of layers in the model
        num_layers = len(self.feature_extractor.layers)
        
        # Create a list of optimizers with different learning rates for each layer
        optimizers = [SGD(learning_rate=lr) for lr in self.learning_rates]
        
        # Set the optimizers for each layer
        for i, optimizer in enumerate(optimizers):
            self.feature_extractor.layers[i].trainable = True
        
        # Compile the model with the final learning rate
        self.feature_extractor.compile(optimizer=optimizers[-1], loss='categorical_crossentropy', metrics=['accuracy'])
    
    def train(self, train_data, train_labels, epochs=10):
        if not self.fine_tuning_started:
            # Perform feature extraction
            optimizer = SGD(learning_rate=self.learning_rates[0])
            self.feature_extractor.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
            self.feature_extractor.fit(train_data, train_labels, epochs=epochs)
            
            # Check the accuracy threshold to start fine-tuning
            accuracy = self.feature_extractor.history.history['accuracy'][-1]
            if accuracy >= 0.6 and accuracy <= 0.7:
                self.fine_tune_model()
                self.fine_tuning_started = True
        else:
            # Fine-tuning
            optimizer = SGD(learning_rate=self.learning_rates[-1])
            self.feature_extractor.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
            self.feature_extractor.fit(train_data, train_labels, epochs=epochs)


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert class labels to one-hot encoded vectors
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define the input shape for the feature extractor
input_shape = x_train.shape[1:]

# Define the learning rates for fine-tuning
learning_rates = [0.001, 0.001, 0.0001]

# Create an instance of FeatureExtractorFineTuner
extractor_fine_tuner = FeatureExtractorFineTuner(input_shape, num_classes, learning_rates)

# Train the model
extractor_fine_tuner.train(x_train, y_train, epochs=20)


29084464/29084464 [==============================] - 1s 0us/step
Epoch 1/20
1563/1563 [==============================] - 41s 16ms/step - loss: 1.9529 - accuracy: 0.3455
Epoch 2/20
1563/1563 [==============================] - 25s 16ms/step - loss: 1.5108 - accuracy: 0.4769
Epoch 3/20
1563/1563 [==============================] - 25s 16ms/step - loss: 1.3894 - accuracy: 0.5188
Epoch 4/20
1563/1563 [==============================] - 25s 16ms/step - loss: 1.3217 - accuracy: 0.5437
Epoch 5/20
1563/1563 [==============================] - 25s 16ms/step - loss: 1.2768 - accuracy: 0.5584
Epoch 6/20
1563/1563 [==============================] - 24s 16ms/step - loss: 1.2463 - accuracy: 0.5692
Epoch 7/20
1563/1563 [==============================] - 24s 16ms/step - loss: 1.2224 - accuracy: 0.5774
Epoch 8/20
1563/1563 [==============================] - 25s 16ms/step - loss: 1.2039 - accuracy: 0.5807
Epoch 9/20
1563/1563 [==============================] - 25s 16ms/step - loss: 1.1899 - accuracy: 0.5887